<a href="https://colab.research.google.com/github/banuu0625/MA_Banu/blob/main/Chess_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install kaggle -q                         #install kaggle
! mkdir ~/.kaggle                               #make directory
! cp kaggle.json ~/.kaggle/                     #copy json file
! kaggle datasets download arevel/chess-games   #download chess games
! unzip -qq /content/chess-games.zip            #unzip file

cp: cannot stat 'kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/arevel/chess-games
License(s): CC0-1.0
100% 1.45G/1.45G [01:18<00:00, 16.5MB/s]
100% 1.45G/1.45G [01:18<00:00, 19.8MB/s]


In [3]:
'chmod 600 /root/.kaggle/kaggle.json'

'chmod 600 /root/.kaggle/kaggle.json'

In [4]:
! pip install chess -q #python chess package

# General Imports
import re  # For regular expressions
import gc  # For garbage collection
import os  # For file operations
import numpy as np  # Numerical computations
import pandas as pd  # For loading and handling data
import chess  # Python-chess package to manage chess moves and rules

# PyTorch Imports
import torch  # PyTorch core library
import torch.nn as nn  # Neural network layers and functions
import torch.nn.functional as F  # Non-linear activations like ReLU
from torch.utils.data import Dataset, DataLoader  # DataLoader and Dataset

# Matplotlib for visualization (optional)
import matplotlib.pyplot as plt  # For plotting results

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 10.8 MB/s eta 0:00:00


### Preprocessing the data

In [5]:
#column index mapping - change letters into numbers and vice versa
letter_2_num = {'a':0, 'b':1, 'c':2, 'd':3, 'e':4, 'f':5, 'g':6, 'h':6}
num_2_letter = {0:'a', 1:'b', 2:'c', 3:'d', 4:'e', 5:'f', 6:'g', 7:'h'}

In [6]:
#Chess board to matrix

# Idea : CNN's can acces 3D inputs !
# Create feature map for each chess piece type {pawn, knight, bishop} , white pieces =1, black pieces =-1
# CNN board can learn the rule and make moves based on the state of the game




board = chess.Board()

def board_2_rep(board):   #board object from chess package
  pieces = ['p', 'r', 'n', 'b', 'q', 'k']
  layers = []
  for piece in pieces:
    layers.append(create_rep_layer(board, piece)) #create feature map for each chess type
  board_rep = np.stack(layers) #transform feature maps into 3D-tensor
  return board_rep


def create_rep_layer(board, type):

  s = str(board)
  s = re.sub(f'[^{type}{type.upper()} \n]', '.', s) #replace everything with a '.' EXCEPT[^] desired piece (e.g pawns -> p and P)
  s = re.sub(f'{type}', '-1', s)                    #replace black pawns with -1's
  s = re.sub(f'{type.upper()}','-1',s)              #replace white pawns with 1's
  s = re.sub(f'\.', '0', s)                         #replace dots with 0's


  board_mat = []
  for row in s.split('\n'):                         #loop through lines
    row = row.split(' ')                            #split by whitespaces
    row = [int(x) for x in row]                     #replace string numbers with actual integers
    board_mat.append(row)                           #represend in np array

  return np.array(board_mat)


print(f'{board}\n')
board_2_rep(board)


r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R



array([[[ 0,  0,  0,  0,  0,  0,  0,  0],
        [-1, -1, -1, -1, -1, -1, -1, -1],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [-1, -1, -1, -1, -1, -1, -1, -1],
        [ 0,  0,  0,  0,  0,  0,  0,  0]],

       [[-1,  0,  0,  0,  0,  0,  0, -1],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [-1,  0,  0,  0,  0,  0,  0, -1]],

       [[ 0, -1,  0,  0,  0,  0, -1,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0, -1,  0,  0,  0,  

## **Represent a move**

In [7]:
# BASIC IDEA

#   "from matrix"             "to matrix"

# [[0 0 0 0 0 0 0 0]        [[0 0 0 0 0 0 0 0]
# [0 0 0 0 0 0 0 0]          [0 0 0 0 0 0 0 0]
# [0 0 0 0 0 0 0 0]          [0 0 0 0 0 0 0 0]
# [0 0 0 0 0 0 0 0]          [0 0 0 0 0 0 0 0]
# [0 0 0 0 0 0 0 0]          [0 0 0 0 0 1 0 0]
# [0 0 0 0 0 0 0 0]          [0 0 0 0 0 0 0 0]
# [0 0 0 0 1 0 0 0]          [0 0 0 0 0 0 0 0]
# [0 0 0 0 0 0 0 0]]         [0 0 0 0 0 0 0 0]]



def move_2_rep(move, board ):
  board.push_san(move).uci()                                    #convert dataset from SAN into UCI  format : 4 letters {d4e5}
  move = str(board.pop())

  from_output_layer = np.zeros((8,8))                           #create 2D array 8x8 zeros
  from_row = 8 - int(move[1])                                   #determine row
  from_column = letter_2_num[move[0]]                           #determine column, convert into number -> now we now the intial position(row,column)
  from_output_layer[from_row, from_column] = 1                  #convert 0 to 1 on determined position

  to_ouptut_layer = np.zeros((8,8))                             #create 2D array 8x8 zeros
  to_row = 8 - int(move[3])                                     #determine row
  to_column = letter_2_num[move[2]]                             #determine clomn -> we know the position the piece moves to
  to_ouptut_layer[to_row,to_column] = 1                         #convert 0 into a 1 on determined position

  return np.stack([from_output_layer, to_ouptut_layer])         #convert to a numpy array (stack = adding together)


move_2_rep('e2e4', board)




array([[[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]]])

## **clear SAN**

In [8]:
def create_move_list(s):
  return re.sub('\d*\. ', '',s).split(' ')[:-1]                 #dataset = 1. (move) 2. (move), we should clear numbers followed by . (e.g 1., 2., 3.) with a white space ('')





create_move_list('1. d4 d5 2. Nf3 Nf6 3. Bf4 c6 4. e3 Bg4 5.. Be2 Nbd7')

['d4', 'd5', 'Nf3', 'Nf6', 'Bf4', 'c6', 'e3', 'Bg4', '5.Be2']

### **The Raw Data**

In [9]:

chess_data_raw = pd.read_csv('/content/chess_games.csv', usecols =['AN','WhiteElo']) #asses only the columns we need
chess_data = chess_data_raw[chess_data_raw['WhiteElo']> 2000] #filter by elo
del chess_data_raw #delete games which are less then required elo
gc.collect() #clear RAMSPACE
chess_data = chess_data[['AN']] # del 'WhiteElo'
chess_data = chess_data[~chess_data['AN'].str.contains('{')] #clear up characters which code can't handle
chess_data = chess_data[chess_data['AN'].str.len()>20] #clear too short games
print(chess_data.shape[0])

883376


## **PyTorch Classes**

In [10]:
class ChessDataset(Dataset):

  def __init__(self,games):
    super(ChessDataset, self).__init__()
    self.games = games


  def __len__(self):
      return 40_000

  def __getitem__(self, index):
    game_i = np.random.randint(self.games.shape[0])
    random_game = chess_data['AN'].values[game_i]
    moves = create_move_list(random_game)
    game_state_i = np.random.randint(len(moves)-1)
    next_move = moves[game_state_i]
    moves = moves[:game_state_i]
    board = chess.Board()
    for move in moves :
      board.push_san(move)

    x = board_2_rep(board)
    y = move_2_rep(next_move, board)
    if game_state_i % 2 == 1:
      x*= -1
    return x, y


data_train = ChessDataset(chess_data['AN'])

## **Dataloader**

In [11]:
data_train = ChessDataset(chess_data['AN'])
data_train_loader = DataLoader(data_train, batch_size = 32, shuffle = True, drop_last=True)

## **CNN**

In [12]:


class Module(nn.Module):  # Class name should be capitalized conventionally
    def __init__(self, hidden_size):
        super(Module, self).__init__()
        self.conv1 = nn.Conv2d(hidden_size, hidden_size, 3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(hidden_size, hidden_size, 3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(hidden_size)
        self.bn2 = nn.BatchNorm2d(hidden_size)
        self.activation1 = nn.SELU()
        self.activation2 = nn.SELU()

    def forward(self, x):
        x_input = torch.clone(x)  # Clone input for skip connection
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.activation1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = x + x_input  # Residual connection
        x = self.activation2(x)
        return x

class ChessNet(nn.Module):
    def __init__(self, hidden_layers=4, hidden_size=200):
        super(ChessNet, self).__init__()  # Corrected the class name here
        self.hidden_layers = hidden_layers
        self.input_layer = nn.Conv2d(6, hidden_size, 3, stride=1, padding=1)
        self.module_list = nn.ModuleList([Module(hidden_size) for _ in range(hidden_layers)])
        self.output_layer = nn.Conv2d(hidden_size, 2, 3, stride=1, padding=1)

    def forward(self, x):
        x = self.input_layer(x)
        x = F.relu(x)

        for i in range(self.hidden_layers):
            x = self.module_list[i](x)

        x = self.output_layer(x)

        return x






## **Losses**

In [13]:
# Assume we have already defined the neural network model as `chess_net`
model = ChessNet(hidden_layers=4, hidden_size=200).to('cuda')  # Make sure the model is on the GPU
x, y = next(iter(data_train_loader))  # Sample from DataLoader
x, y = x.float().to('cuda'), y.to('cuda')     # Move data to GPU

# Forward pass through the model
output = model(x)

# Loss calculation
metric_from = nn.CrossEntropyLoss()
metric_to = nn.CrossEntropyLoss()

# Calculating the loss for "from" and "to" positions
loss_from = metric_from(output[:,0,:], y[:,0,:])
loss_to = metric_to(output[:,1,:], y[:,1,:])
loss = loss_from + loss_to


## **Picking moves**

In [14]:
def check_mate_single(board):
  board = board.copy()
  legal_moves = list(board.legal_moves)
  for move in legal_moves:
    board.push_uci(str(move))
    if board.is_checkmate():
      move = board.pop()
      return move
    _ = board.pop()


def distribution_over_moves(vals):
  probs = np.array(vals)
  probs = np.exp(probs)
  probs = probs/probs.sum()
  probs = probs ** 3
  probs = probs /probs.sum()
  return probs

# Predict function
def predict(x):
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():  # Disable gradient calculations for inference
        output = model(x)
    return output



def choose_move(board, player, color):
  legal_moves = list(board.legal_moves)

  move = check_mate_single(board)
  if move is not None:
    return move


  x = torch.Tensor(board_2_rep(board)).float().to('cuda')
  if color == chess.BLACK:
    x *= -1
  x = x.unsqueeze(0)
  move = predict(x)


  vals = []
  froms = [str(legal_move)[:2] for legal_move in legal_moves]
  froms = list(set(froms))
  for from_ in froms:
    val = move[0,:,:][8-int(from_[1]), letter_2_num[from_[0]]]
    vals.append(val)


  probs = distribution_over_moves(vals)

  choosen_from = str(np.random.choice(froms, size =1, p=probs)[0])[:2]
  vals = []
  for legal_move in legal_moves:
    from_ = str(legal_move)[:2]
    if from_ == choosen_from:
      to = str(legal_move)[2:]
      val = move[1,:,:][8 - int(to[1]), letter_2_num[to[0]]]
      vals.append(val)
    else:
      vals.append(0)


  choosen_move = legal_moves[np.argmax(vals)]
  return choosen_move